In [1]:
import openrouteservice
import pandas as pd
import folium
import pickle
pd.set_option('display.max_rows', 100)

In [2]:
client = openrouteservice.Client(key='5b3ce3597851110001cf62485a0e241a183542b1b686d0d9183a50a9')

In [3]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [4]:
df = pd.read_csv('LocationDifferentials2021.csv',header=0)
df = df.loc[df['Port']==df['NTP']]
df.set_index('Location', inplace=True)
#remove GRENFELL entry
df.drop('GRENFELL', inplace=True)

In [5]:
locations = [(f"{df.index[x]}, {df['State'].iloc[x]}", df['LD'].iloc[x]) for x in range(len(df))]
locations_set = list(set(locations))

In [6]:
len(locations_set)

456

In [7]:
#find locations with same site names but different NTP differentials
for i in range(len(df)-1):
    if (df.index[i], df['State'][i]) == (df.index[i+1], df['State'][i+1]) and (df['LD'][i] != df['LD'][i+1]):
        print(df.index[i], df['State'][i], df['LD'][i])

HAY NSW 40.75
YENDA NSW 47.0
BOORT VIC 27.75
LOCK SA 13.55


In [74]:
distance={}
demandList = []

#convert demand dictionary into list of just dict values
for value in demand.values():
    demandList.append(list(reversed(value)))
    
for key, value in coords.items():
    demandTemp = demandList.copy()
    demandTemp.insert(0,list(reversed(value)))
    distance[key] = client.distance_matrix(locations=demandTemp, profile='driving-hgv', sources=[0], destinations=list(range(1,len(demandTemp))), metrics=['distance'], units='km')

/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openroutes

In [75]:
#with open('./pickle/distanceMatrixHGV.pickle', 'wb') as handle:
#    pickle.dump(distance, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
#import location coordinates
with open('./pickle/coords.pickle', 'rb') as handle:
    coords = pickle.load(handle)

In [9]:
#import demand location coords
with open('./pickle/demand.pickle' ,'rb') as handle:
    demand = pickle.load(handle)

In [10]:
with open('./pickle/distanceMatrixHGV.pickle', 'rb') as handle:
    distanceMatrix = pickle.load(handle)

In [80]:
distanceSummary = {}
for sourceName, distanceValue in distanceMatrix.items():
    distanceSummary[sourceName] = {}
    distanceSummary[sourceName]['source'] = (sourceName, distanceValue['sources'][0]['location'], distanceValue['metadata']['query']['locations'][0])
    distanceSummary[sourceName]['destinations'] = []
    for demandName, demandCoord in demand.items():
        for queriedCoord in distanceValue['metadata']['query']['locations']:
            #find the matching location name for the queried coordinates
            numIndex = distanceValue['metadata']['query']['locations'].index(queriedCoord)
            destTemp = distanceValue['destinations'][numIndex-1]['location']    #numIndex-1 as the queriedCoord has 1 extra entry for the source location
            distanceTemp = distanceValue['distances'][0][numIndex-1]    #find the distance
            
            #freight rate for various km trips
            if distanceTemp < 50:
                freight = 10
            elif 50 <= distanceTemp < 100:
                freight = 15
            elif 100 <= distanceTemp < 140:
                freight = 18
            elif 140 <= distanceTemp < 180:
                freight = distanceTemp * 0.13
            elif 180 <= distanceTemp < 400:
                freight = distanceTemp * 0.12
            elif 400 <= distanceTemp < 500:
                freight = distanceTemp * 0.115
            elif 500 <= distanceTemp < 700:
                freight = distanceTemp * 0.11
            elif 700 <= distanceTemp < 800:
                freight = distanceTemp * 0.105
            elif 800 <= distanceTemp < 1000:
                freight = distanceTemp * 0.1
            elif 1000 <= distanceTemp:
                freight = distanceTemp * 0.095
            
            #add entry to distanceSummary if the queried coordinates match the demand coordinate so the correct demand name is input
            if list(reversed(demandCoord)) == queriedCoord:
                distanceSummary[sourceName]['destinations'].append([demandName, destTemp, queriedCoord, distanceTemp, round(freight,0)])

In [81]:
#with open('./pickle/distanceSummaryHGV.pickle','wb') as handle:
#    pickle.dump(distanceSummary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open('./pickle/distanceSummaryHGV.pickle','rb') as handle:
    distanceSummary = pickle.load(handle)

In [53]:
distanceSummary['BELLATA, NSW']['destinations'][0]

['FISHERMAN ISLAND, QLD',
 [153.167759, -27.383615],
 [153.16773624319558, -27.383617388670885],
 553.17,
 61.0]

In [13]:
freightTable = {}
colNames = [x[0] for x in distanceSummary['MALU, QLD']['destinations']]
for sourceName, distanceDict in distanceSummary.items():
    freightTable[sourceName] = []
    for lists in distanceDict['destinations']:
        freightTable[sourceName].append(lists[4])

In [54]:
distanceTable = {}
colNames = [x[0] for x in distanceSummary['MALU, QLD']['destinations']]
for sourceName, distanceDict in distanceSummary.items():
    distanceTable[sourceName] = []
    for lists in distanceDict['destinations']:
        distanceTable[sourceName].append(lists[3])

In [14]:
demand.keys()

dict_keys(['FISHERMAN ISLAND, QLD', 'GLADSTONE, QLD', 'MACKAY, QLD', 'NARANGBA, QLD', 'TOOWOOMBA, QLD', 'CLIFTON, QLD', 'CONDAMINE, QLD', 'DALBY, QLD', 'RANGERS VALLEY, NSW', 'TEXAS, QLD', 'QUIRINDI, NSW', 'TAMWORTH, NSW', 'TAREE, NSW', 'NEWCASTLE, NSW', 'MANILDRA, NSW', 'NOWRA, NSW', 'YOUNG, NSW', 'HANWOOD, NSW', 'PORT KEMBLA, NSW', 'COROWA, NSW', 'PEECHELBA, VIC', 'COLBINABBIN, VIC', 'KYABRAM, VIC', 'NATHALIA, VIC', 'CHARLTON, VIC', 'GUNBOWER, VIC', 'WELLSFORD, VIC', 'ST.ARNAUD, VIC', 'LARA, VIC', 'PAKENHAM, VIC', 'MAFFRA, VIC', 'TERANG, VIC', 'BRIDGEWATER, VIC', 'GEELONG, VIC', 'PORTLAND, VIC', 'MURRAY BRIDGE, SA', 'WASLEYS, SA', 'ADELAIDE, SA', 'WALLAROO, SA', 'PORT GILES, SA', 'PORT LINCOLN, SA', 'THEVENARD, SA', 'ESPERANCE, WA', 'ALBANY, WA', 'KWINANA, WA', 'GERALDTON, WA', 'MELBOURNE, VIC'])

In [58]:
freight_df = pd.DataFrame.from_dict(freightTable,orient='index', columns=colNames)

In [55]:
distance_df = pd.DataFrame.from_dict(distanceTable,orient='index',columns=colNames)

In [56]:
distance_df

,"FISHERMAN ISLAND, QLD","GLADSTONE, QLD","MACKAY, QLD","NARANGBA, QLD","TOOWOOMBA, QLD","CLIFTON, QLD","CONDAMINE, QLD","DALBY, QLD","RANGERS VALLEY, NSW","TEXAS, QLD",...,"ADELAIDE, SA","WALLAROO, SA","PORT GILES, SA","PORT LINCOLN, SA","THEVENARD, SA","ESPERANCE, WA","ALBANY, WA","KWINANA, WA","GERALDTON, WA","MELBOURNE, VIC"
"MANMANNING, WA",4403.22,4796.45,5039.46,4412.50,4240.87,4235.43,4293.47,4316.76,4113.72,4115.33,...,2659.72,2583.56,2687.68,2314.23,1895.82,900.05,560.80,249.41,427.11,3388.16
"BARELLAN, NSW",1236.80,1630.03,1873.04,1246.08,1074.45,1069.01,1127.05,1150.34,947.30,948.91,...,910.07,1013.40,1077.23,1528.27,1632.22,3033.64,3508.81,3585.46,3935.30,491.84
"LILLIMUR, VIC",2056.49,2449.72,2692.74,2065.77,1894.14,1888.70,1946.74,1970.03,1912.43,1768.61,...,324.58,466.90,530.73,981.77,1085.72,2487.14,2962.30,3038.95,3388.79,415.66
"NANDALY, VIC",1811.81,2205.04,2448.05,1821.09,1649.46,1644.02,1702.06,1725.35,1522.31,1523.92,...,544.14,647.47,711.30,1162.34,1266.29,2667.71,3142.87,3219.52,3569.36,385.95
"TOMINGLEY WEST, NSW",937.96,1331.19,1574.20,947.24,775.61,770.17,828.21,851.50,648.46,650.07,...,1199.53,1254.75,1366.69,1538.45,1642.40,3043.82,3518.98,3595.63,3945.47,781.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WALGETT, NSW",715.19,1108.41,1278.87,724.47,552.84,547.39,605.44,628.73,462.70,427.30,...,1333.46,1319.20,1414.58,1602.90,1706.85,3108.27,3583.43,3660.08,4009.92,1103.00
"CUNNINGAR, NSW",1245.17,1593.40,1836.42,1266.71,1037.82,1032.38,1090.42,1113.71,975.79,912.29,...,1082.24,1185.58,1249.40,1700.45,1804.40,3205.81,3680.98,3757.63,4107.47,581.50
"HOGAN, NSW",1378.75,1771.98,2015.00,1388.03,1216.40,1210.96,1269.00,1292.29,1089.25,1090.87,...,956.58,1059.91,1123.74,1574.78,1678.73,3080.15,3555.31,3631.97,3981.81,325.80
"MT.MCLAREN, QLD",1091.22,552.97,273.69,1047.29,928.22,983.58,765.07,851.94,1213.19,1037.53,...,2430.75,2416.49,2511.87,2700.19,2804.14,4205.55,4680.72,4757.37,5107.21,2280.06


In [17]:
prices = pd.read_csv('./prices.csv',index_col='LOCATION').T

In [23]:
depot = -freight_df.sub(prices.loc['ASW'], axis='columns')-18

In [24]:
depot['MAFFRA, VIC'].sort_values().tail(10)

ELMORE, VIC            279.0
TANDARA, VIC           279.0
DUNOLLY, VIC           280.0
LISMORE, VIC           281.0
BRIDGEWATER, VIC       281.0
MURCHISON EAST, VIC    282.0
BERRYBANK, VIC         283.0
WERNETH, VIC           284.0
BALLIANG, VIC          293.0
LARA, VIC              294.0
Name: MAFFRA, VIC, dtype: float64

In [25]:
df_location = pd.DataFrame.from_records(locations, columns=['Location','LD'])
df_location.set_index('Location',inplace=True)

In [30]:
df_location_mod = df_location.copy()

In [31]:
df_location_mod = df_location_mod.groupby('Location').mean()

In [32]:
track = depot.add(df_location_mod['LD'], axis='index')

In [33]:
track['MAFFRA, VIC'].sort_values().tail(10)

DUNOLLY, VIC           301.75
BROCKLESBY, NSW        301.75
BLIGHTY, NSW           301.75
MURCHISON EAST, VIC    301.75
NARRANDERA, NSW        302.00
YANCO, NSW             302.25
BERRIGAN, NSW          302.30
BALLIANG, VIC          303.00
MERRIWAGGA, NSW        303.25
LARA, VIC              304.00
Name: MAFFRA, VIC, dtype: float64

In [59]:
maxValues = {}
for column in track.T.columns:
    sourceMax = track.T[column].idxmax()
    MaxValue = track.T[column].max()
    maxValues[column] = (sourceMax, MaxValue, freight_df.loc[column][sourceMax], distance_df.loc[column][sourceMax])

In [60]:
maxValues

{'AINSWORTH, WA': ('KWINANA, WA', 328.86, 31.0, 259.86),
 'ALBANY TERMINAL, WA': ('ALBANY, WA', 322.0, 10.0, 0.01),
 'ALDERSYDE, WA': ('KWINANA, WA', 327.72, 22.0, 181.97),
 'ALLORA, QLD': ('FISHERMAN ISLAND, QLD', 334.75, 22.0, 180.44),
 'APAMURRA, SA': ('ADELAIDE, SA', 338.37, 15.0, 78.84),
 'ARDATH, WA': ('KWINANA, WA', 318.55, 38.0, 319.93),
 'ARDLETHAN, NSW': ('MELBOURNE, VIC', 308.75, 55.0, 502.3),
 'ARDROSSAN, SA': ('ADELAIDE, SA', 327.25, 19.0, 143.69),
 'ARIAH PARK, NSW': ('PORT KEMBLA, NSW', 313.25, 48.0, 415.97),
 'ARMATREE, NSW': ('QUIRINDI, NSW', 310.5, 33.0, 275.3),
 'ARNO BAY, SA': ('PORT LINCOLN, SA', 325.98, 18.0, 118.24),
 'ARRINO, WA': ('GERALDTON, WA', 324.96, 20.0, 154.41),
 'ARTHUR RIVER, WA': ('KWINANA, WA', 321.95, 25.0, 205.54),
 'AVON, WA': ('KWINANA, WA', 323.41, 18.0, 134.49),
 'BALAKLAVA, SA': ('WASLEYS, SA', 340.72, 10.0, 48.02),
 'BALLIANG, VIC': ('GEELONG, VIC', 319.0, 10.0, 30.87),
 'BALLIDU, WA': ('KWINANA, WA', 316.44, 36.0, 298.88),
 'BARADINE, NSW':

In [35]:
df_location.loc['LARA, VIC']

LD    10.0
Name: LARA, VIC, dtype: float64

In [36]:
freight_df['PAKENHAM, VIC'].loc['LARA, VIC']

18.0

In [37]:
track_murraybridge = depot['MURRAY BRIDGE, SA'].add(df_location['LD'])
track_wasley = depot['WASLEYS, SA'].add(df_location['LD'])

In [38]:
track_wasley.max()

347.5

In [39]:
df_location.loc['YELTA, VIC']
df_location.loc['LOXTON, SA']

LD    25.6
Name: LOXTON, SA, dtype: float64

In [123]:
m = folium.Map(location=list(reversed([117.1, -30.85])), tiles='cartodbpositron', zoom_start=13)
basemaps['Google Satellite Hybrid'].add_to(m)

In [124]:
for key,Value in maxValues.items():
    sourceCoord = coords[key]
    destinationCoord = demand[Value[0]]
    folium.PolyLine(locations=[sourceCoord,destinationCoord], color='red', weight=3, opacity=1).add_to(m)

In [125]:

for key, value in coords.items():
    popup_content = {'source': key, 'dest': maxValues[key][0], 'distance': maxValues[key][3], 'freight': maxValues[key][2]}
    html = "{source}<br>dest: {dest}<br>distance: {distance}km<br>freight: ${freight}".format(**popup_content)
    iframe = folium.IFrame(html,
                       width=250,
                       height=100)
    folium.Marker(
        location=list(value),
        icon = folium.Icon(icon='compass', color='red', prefix='fa'),
        popup=folium.Popup(iframe)).add_to(m)

In [126]:
m